# AGStream Complete Tutorial with Schema Registry

This interactive tutorial demonstrates the complete AGStream workflow with Karapace Schema Registry.

## Prerequisites

Before starting, make sure you have:
1. Docker running with Kafka and Karapace services
2. Python environment with agentics installed
3. Basic understanding of Pydantic models

```bash
# Start services (if not already running)
./manage_services.sh start
```

## 1. Setup and Imports

Let's import the necessary modules and set up our environment.

Import required modules

In [ ]:
from pydantic import BaseModel, Field
from agentics.core.streaming import AGStream
from agentics.core.streaming_utils import (
    register_atype_schema,
    get_atype_from_registry,
    list_schema_versions,
    create_kafka_topic,
    kafka_topic_exists,
)
import json
import time
import uuid
import threading
import queue

# Set up logging for better visibility
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Global queue for communication between threads
message_queue = queue.Queue()

## 2. Define Pydantic Models

Let's create some Pydantic models that we'll use throughout this tutorial.

Define a Question model

In [ ]:
class Question(BaseModel):
    """A question to be answered by our agent"""
    text: str = Field(description="The question text")
    category: str = Field(description="Question category")
    priority: int = Field(default=1, description="Priority level (1-5)")

# Define an Answer model
class Answer(BaseModel):
    """An answer to a question"""
    text: str = Field(description="The answer text")
    confidence: float = Field(description="Confidence score (0-1)")
    sources: list[str] = Field(default_factory=list, description="Source references")

# Display our models
print("Question model fields:", list(Question.model_fields.keys()))
print("Answer model fields:", list(Answer.model_fields.keys()))

## 3. Initialize AGStream with Schema Registry

Let's create AGStream instances and connect them to the schema registry.

Create AGStream for questions

In [ ]:
question_stream = AGStream(
    atype=Question,
    input_topic="tutorial-questions",
    output_topic="tutorial-answers",
    schema_registry_url="http://localhost:8081"
)

# Create AGStream for answers
answer_stream = AGStream(
    atype=Answer,
    input_topic="tutorial-answers",
    output_topic="tutorial-questions",
    schema_registry_url="http://localhost:8081"
)

print(f"Question stream configured for topic: {question_stream.input_topic}")
print(f"Answer stream configured for topic: {answer_stream.input_topic}")

## 4. Register Schemas in Schema Registry

Let's register our Pydantic models as JSON Schemas in the schema registry.

Register Question schema

In [ ]:
question_schema_id = register_atype_schema(atype=Question, schema_registry_url="http://localhost:8081")
print(f"✅ Question schema registered with ID: {question_schema_id}")

# Register Answer schema
answer_schema_id = register_atype_schema(atype=Answer, schema_registry_url="http://localhost:8081")
print(f"✅ Answer schema registered with ID: {answer_schema_id}")

# List registered schemas
question_versions = list_schema_versions(atype_name="Question", schema_registry_url="http://localhost:8081")
answer_versions = list_schema_versions(atype_name="Answer", schema_registry_url="http://localhost:8081")

print(f"\nQuestion schema versions: {question_versions}")
print(f"Answer schema versions: {answer_versions}")

## 5. Create Kafka Topics

Before we can produce and consume messages, we need to create the Kafka topics.

Create topics if they don't exist

In [ ]:
for topic in ["tutorial-questions", "tutorial-answers"]:
    if not kafka_topic_exists(topic):
        create_kafka_topic(topic)
        print(f"✅ Created topic: {topic}")
        time.sleep(1)  # Give Kafka time to create the topic
    else:
        print(f"✅ Topic already exists: {topic}")

print("\nAll topics are ready!")

## 6. Start Listener in Background

Let's set up a listener that will process questions and generate answers.

Create a listener stream

In [ ]:
listener = AGStream(
    atype=Answer,
    input_topic="tutorial-questions",
    output_topic="tutorial-answers",
    schema_registry_url="http://localhost:8081"
)

# Start the listener in a separate thread
def listener_thread():
    print("\n🎧 Starting listener...")
    try:
        # Listen for questions and produce answers
        listener.listen(source_atype_name="Question")
    except Exception as e:
        print(f"Listener error: {e}")
        message_queue.put(f"Listener error: {e}")

# Start listener in background
listener_thread = threading.Thread(target=listener_thread, daemon=True)
listener_thread.start()

print("Listener started in background. It will process questions and generate answers.")
print("Waiting 5 seconds for listener to initialize...")
time.sleep(5)  # Give listener time to start

## 7. Produce and Consume Messages

Now let's create a complete workflow: produce questions, let the listener process them, and then consume the answers.

Create some questions

In [ ]:
questions = [
    Question(text="What is machine learning?", category="AI", priority=3),
    Question(text="How does blockchain work?", category="Technology", priority=2),
    Question(text="What is the capital of France?", category="Geography", priority=2)
]

# Function to send questions and collect answers
def send_and_collect():
    # Send questions to Kafka
    question_keys = []
    for question in questions:
        question_stream.states = [question]
        msg_ids = question_stream.produce(register_if_missing=True)
        key = msg_ids[0] if msg_ids else None
        question_keys.append(key)
        print(f"✅ Sent question: {question.text[:30]}... (key: {key})")
        time.sleep(1)  # Small delay between messages
    
    # Wait for processing
    print("\n🕒 Waiting for answers to be processed...")
    time.sleep(5)  # Give listener time to process
    
    # Create answer collector
    answer_collector = AGStream(
        input_topic="tutorial-answers",
        schema_registry_url="http://localhost:8081"
    )
    
    # Retrieve Answer type from registry
    RetrievedAnswer = get_atype_from_registry(atype_name="Answer", schema_registry_url="http://localhost:8081")
    if RetrievedAnswer:
        answer_collector.atype = RetrievedAnswer
        print(f"✅ Retrieved Answer type from registry: {RetrievedAnswer.__name__}")
    
    # Collect answers
    answers = answer_collector.collect_sources(
        max_messages=5,
        timeout_ms=10000,  # Longer timeout
        mode='latest',
        verbose=True
    )
    
    print(f"\n📋 Collected {len(answers)} answers:")
    for i, answer in enumerate(answers, 1):
        if answer.states:
            print(f"{i}. {answer.states[0].text[:50]}... (Confidence: {answer.states[0].confidence})")
    
    return answers

# Run the workflow
answers = send_and_collect()

## 8. Schema Evolution Example

Let's demonstrate how schema evolution works with the registry.

Define an updated version of our Question model

In [ ]:
class QuestionV2(BaseModel):
    """Updated question model with additional fields"""
    text: str = Field(description="The question text")
    category: str = Field(description="Question category")
    priority: int = Field(default=1, description="Priority level (1-5)")
    language: str = Field(default="en", description="Language of the question")
    timestamp: float = Field(default_factory=time.time, description="When the question was asked")

# Create a new stream with the updated type
updated_stream = AGStream(
    atype=QuestionV2,
    input_topic="tutorial-questions",
    schema_registry_url="http://localhost:8081"
)

# Register the updated schema
updated_schema_id = register_atype_schema(atype=QuestionV2, schema_registry_url="http://localhost:8081")
print(f"✅ Updated Question schema registered with ID: {updated_schema_id}")

# List all versions
all_versions = list_schema_versions(atype_name="QuestionV2", schema_registry_url="http://localhost:8081")
print(f"All Question schema versions: {all_versions}")

# Retrieve version 1 (original)
OriginalQuestion = get_atype_from_registry(atype_name="QuestionV2", schema_registry_url="http://localhost:8081", version="1")
print(f"Original Question type: {OriginalQuestion.__name__ if OriginalQuestion else 'Not found'}")

# Retrieve latest version
LatestQuestion = get_atype_from_registry(atype_name="QuestionV2", schema_registry_url="http://localhost:8081", version="latest")
print(f"Latest Question type: {LatestQuestion.__name__ if LatestQuestion else 'Not found'}")

# Show the difference
if OriginalQuestion and LatestQuestion:
    print("\n🔄 Schema Evolution:")
    orig_fields = set(OriginalQuestion.model_fields.keys())
    latest_fields = set(LatestQuestion.model_fields.keys())
    print(f"   Original fields: {orig_fields}")
    print(f"   Latest fields: {latest_fields}")
    print(f"   Added fields: {latest_fields - orig_fields}")

## 9. Direct Schema Registry Access

Let's explore the schema registry directly using the REST API.

In [ ]:
import requests

# List all subjects
def list_subjects():
    try:
        response = requests.get("http://localhost:8081/subjects")
        if response.status_code == 200:
            subjects = response.json()
            print("📋 Registered Subjects:")
            for subject in subjects:
                print(f"   - {subject}")
            return subjects
        else:
            print(f"❌ Error listing subjects: {response.status_code} - {response.text}")
            return []
    except Exception as e:
        print(f"❌ Error: {e}")
        return []

# Get schema details
def get_schema_details(subject: str, version: str = "latest"):
    try:
        url = f"http://localhost:8081/subjects/{subject}/versions/{version}"
        response = requests.get(url)
        if response.status_code == 200:
            details = response.json()
            print(f"\n📄 Schema Details for {subject} (version {version}):")
            print(f"   ID: {details.get('id')}")
            print(f"   Version: {details.get('version')}")
            print(f"   Schema: {details.get('schema')[:100]}...")
            return details
        else:
            print(f"❌ Error getting schema: {response.status_code} - {response.text}")
            return None
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

# List all subjects
subjects = list_subjects()

# Get details for each subject
if subjects:
    for subject in subjects:
        get_schema_details(subject)
        time.sleep(0.5)  # Small delay between requests

## 10. Advanced: Schema Validation

Let's demonstrate how schema validation works.

In [ ]:
from agentics.core.streaming_utils import get_schema_info

# Validate a message dict against a registered schema using streaming_utils.
# The subject name is derived from the Pydantic class name (e.g. 'Question-value'),
# NOT from the Kafka topic name.
def validate_message(message: dict, atype_name: str,
                     schema_registry_url: str = "http://localhost:8081") -> bool:
    """Validate a message dict against a schema fetched from the registry."""
    info = get_schema_info(
        atype_name=atype_name,
        schema_registry_url=schema_registry_url,
        is_key=False,
        version="latest",
        add_suffix=True,
    )
    if info is None:
        print(f"❌ Schema not found for '{atype_name}-value'")
        return False

    schema = info["schema"]
    required_fields = schema.get("required", [])

    for field in required_fields:
        if field not in message:
            print(f"❌ Missing required field: {field}")
            return False

    print(f"✅ Message is valid against schema '{info['subject']}' (ID: {info['id']})")
    return True

# Test validation with a valid message
# Note: subject is derived from the class name 'Question', not the topic name.
test_message = {
    "text": "Test question",
    "category": "Test",
    "priority": 1
}

print("\n🔍 Testing message validation:")
is_valid = validate_message(test_message, "Question")
print(f"Validation result: {'✅ PASS' if is_valid else '❌ FAIL'}")

# Test with invalid message (missing required field)
invalid_message = {
    "text": "Invalid question",
    "priority": 1
    # Missing 'category' which is required
}

print("\n🔍 Testing invalid message:")
is_valid = validate_message(invalid_message, "Question")
print(f"Validation result: {'✅ PASS' if is_valid else '❌ FAIL'}")

## 11. Cleanup and Summary

Let's clean up our resources and summarize what we've learned.

In [ ]:
print("\n🧹 Cleanup complete!")
print("\n📚 Summary of what we've learned:")
print("\n1. ✅ Schema Registration: Stored Pydantic models as JSON Schemas")
print("2. ✅ Schema Retrieval: Dynamically loaded types from registry")
print("3. ✅ Message Production: Sent type-safe messages to Kafka")
print("4. ✅ Message Consumption: Retrieved messages using registered schemas")
print("5. ✅ Listener Setup: Processed messages with schema validation")
print("6. ✅ Schema Evolution: Managed versioning and compatibility")
print("7. ✅ Direct API Access: Explored schema registry REST API")
print("8. ✅ Schema Validation: Validated messages against schemas")

print("\n🎯 Key Benefits:")
print("- Type Safety: Ensure messages conform to registered schemas")
print("- Schema Evolution: Track changes and maintain compatibility")
print("- Dynamic Discovery: Load types at runtime without hardcoding")
print("- Centralized Management: Store all schemas in one place")

print("\n🚀 Next Steps:")
print("- Explore Schema Registry UI: http://localhost:8000")
print("- Check Kafka UI: http://localhost:8080")
print("- Try creating your own models and integrating them")
print("- Experiment with more complex schema evolution scenarios")
print("- Integrate with your production streaming pipelines")

## 12. `listen` — LLM Transduction on the Stream

The `listen` method combines schema-validated Kafka consumption
with **logical transduction**: for every incoming state it calls `self << source` (the
AGStream `__lshift__` operator) which invokes the configured LLM to convert the source
object into the target type, then produces the result back to the output topic.

### Workflow

```
input_topic  ──►  validate (schema registry)  ──►  transduce (LLM)  ──►  output_topic
  Question                                           Question → MovieSummary
```

### Key parameters

| Parameter | Default | Description |
|---|---|---|
| `source_atype_name` | `None` | Fetch source type from registry by name |
| `validate_schema` | `True` | Reject states that fail Pydantic validation |
| `produce_results` | `True` | Write transduced states to `output_topic` |
| `verbose` | `False` | Print per-state progress |
| `group_id` | auto UUID | Kafka consumer group (unique → always reads from earliest) |

### Step 1: Define source and target Pydantic models

In [ ]:
from pydantic import BaseModel, Field
from agentics.core.streaming import AGStream
import time
import threading

class MovieReview(BaseModel):
    """A raw movie review as it arrives on the stream."""
    title: str = Field(description="Movie title")
    review: str = Field(description="Full review text")
    rating: float = Field(description="Numeric rating 1-10")

class MovieSummary(BaseModel):
    """A concise summary produced by the LLM transducer."""
    title: str = Field(description="Movie title")
    one_line_summary: str = Field(description="One-sentence summary of the review")
    sentiment: str = Field(description="Overall sentiment: positive / neutral / negative")

print("MovieReview fields :", list(MovieReview.model_fields.keys()))
print("MovieSummary fields:", list(MovieSummary.model_fields.keys()))

### Step 2: Create Kafka topics

In [ ]:
for topic in ["movie-reviews", "movie-summaries"]:
    if not kafka_topic_exists(topic):
        create_kafka_topic(topic)
        print(f"✅ Created topic: {topic}")
        time.sleep(1)
    else:
        print(f"✅ Topic already exists: {topic}")

### Step 3: Register schemas in the schema registry

In [ ]:
review_stream = AGStream(
    atype=MovieReview,
    input_topic="movie-reviews",
    output_topic="movie-summaries",
    schema_registry_url="http://localhost:8081",
)

summary_stream = AGStream(
    atype=MovieSummary,
    input_topic="movie-summaries",
    output_topic="movie-summaries",
    schema_registry_url="http://localhost:8081",
)

review_schema_id  = register_atype_schema(atype=MovieReview, schema_registry_url="http://localhost:8081")
summary_schema_id = register_atype_schema(atype=MovieSummary, schema_registry_url="http://localhost:8081")

print(f"✅ MovieReview  schema registered (ID: {review_schema_id})")
print(f"✅ MovieSummary schema registered (ID: {summary_schema_id})")

### Step 4: Build the transducing listener

The listener AGStream is configured with:
  • atype          = MovieSummary  (the TARGET type the LLM must produce)
  • input_topic    = "movie-reviews"  (where raw reviews arrive)
  • output_topic   = "movie-summaries"  (where summaries are written)
  • instructions   = task description passed to the LLM

listen() will:
  1. Poll "movie-reviews" continuously
  2. Validate each incoming state as a MovieReview
  3. Call  listener << source_ag  (LLM transduction)
  4. Produce the resulting MovieSummary to "movie-summaries"

In [ ]:
listener = AGStream(
    atype=MovieSummary,
    input_topic="movie-reviews",
    output_topic="movie-summaries",
    schema_registry_url="http://localhost:8081",
    instructions=(
        "You receive a movie review. "
        "Produce a MovieSummary with a one-sentence summary and the overall sentiment."
    ),
)

print("Listener configured:")
print(f"  input_topic  : {listener.input_topic}")
print(f"  output_topic : {listener.output_topic}")
print(f"  target type  : {listener.atype.__name__}")
print(f"  instructions : {listener.instructions}")

### Step 5: Ensure schemas are registered, then start the listener

IMPORTANT: The listener fetches the source schema ('MovieReview-value') from
the schema registry on startup.  If Step 3 was skipped or the registry was
not yet ready, the fetch would fail with a 404.  We guard against this by
(a) registering both schemas here if they are missing, and
(b) passing schema_fetch_retries so the listener retries a few times with
    a short delay before giving up.

Guard: register schemas if they haven't been registered yet

In [ ]:
for atype, name in [(MovieReview, 'MovieReview'), (MovieSummary, 'MovieSummary')]:
    versions = list_schema_versions(atype_name=name, schema_registry_url="http://localhost:8081")
    if not versions:
        schema_id = register_atype_schema(atype=atype, schema_registry_url="http://localhost:8081")
        print(f"✅ Registered {name} schema (ID: {schema_id})")
    else:
        print(f"✅ {name} schema already registered (versions: {versions})")

def run_listener():
    try:
        listener.listen(
            source_atype_name="MovieReview",  # fetch source type from registry
            validate_schema=True,
            produce_results=True,
            verbose=True,
            schema_fetch_retries=5,        # retry up to 5 times if schema not found
            schema_fetch_retry_delay=2.0,  # wait 2 s between retries
        )
    except Exception as exc:
        print(f"Listener thread exited with error: {exc}")

listener_thread = threading.Thread(target=run_listener, daemon=True)
listener_thread.start()

print("🎧 Listener thread started. Waiting 3 s for it to initialise...")
time.sleep(3)

### Step 6: Produce some MovieReview messages

In [ ]:
sample_reviews = [
    MovieReview(
        title="Inception",
        review=(
            "A mind-bending thriller that challenges the boundaries of reality. "
            "Nolan's direction is masterful and the performances are outstanding."
        ),
        rating=9.2,
    ),
    MovieReview(
        title="The Room",
        review=(
            "Widely considered one of the worst films ever made. "
            "The dialogue is stilted, the plot makes no sense, and the acting is painful."
        ),
        rating=1.8,
    ),
    MovieReview(
        title="Parasite",
        review=(
            "A brilliant social commentary wrapped in a gripping thriller. "
            "Bong Joon-ho delivers a near-perfect film that deserved every Oscar it won."
        ),
        rating=9.5,
    ),
]

producer_stream = AGStream(
    atype=MovieReview,
    input_topic="movie-reviews",
    schema_registry_url="http://localhost:8081",
)
producer_stream.states = sample_reviews

msg_ids = producer_stream.produce(register_if_missing=True)
print(f"\n✅ Produced {len(msg_ids)} MovieReview messages to 'movie-reviews'")

### Step 7: Wait for transduction, then collect results

In [ ]:
print("⏳ Waiting 15 s for the listener to transduce all reviews...")
time.sleep(15)

# Collect the MovieSummary objects from the output topic
collector = AGStream(
    atype=MovieSummary,
    input_topic="movie-summaries",
    schema_registry_url="http://localhost:8081",
)

results = collector.collect_sources(
    mode="all",
    max_messages=20,
    timeout_ms=8000,
    validate_schema=True,
    verbose=True,
)

print(f"\n📋 Collected {len(results)} MovieSummary objects from 'movie-summaries':")
for i, ag in enumerate(results, 1):
    if ag.states:
        s = ag.states[0]
        print(f"\n  [{i}] {s.title}")
        print(f"       Summary  : {s.one_line_summary}")
        print(f"       Sentiment: {s.sentiment}")

### Step 8: Demonstrate produce_results=False (dry-run: transduce but don't write)

Sometimes you want to transduce a single message and inspect the result
without writing it back to Kafka.  Set produce_results=False and the
listener will still transduce but skip the produce step.

Key settings for a clean dry-run:
  • auto_offset_reset='latest'  — only process messages produced AFTER the
                                  listener starts; avoids replaying old messages
  • max_empty_polls=5           — exit automatically once the topic goes quiet
  • produce_results=False       — transduce locally, nothing written to Kafka

8a: Produce one fresh review BEFORE starting the listener

In [ ]:
extra_review = MovieReview(
    title="Everything Everywhere All at Once",
    review=(
        "An absurdist multiverse adventure that somehow manages to be deeply emotional. "
        "A genre-defying masterpiece."
    ),
    rating=9.0,
)
dry_run_producer = AGStream(
    atype=MovieReview,
    input_topic="movie-reviews",
    schema_registry_url="http://localhost:8081",
)
dry_run_producer.states = [extra_review]
dry_run_producer.produce(register_if_missing=True)
print("✅ Extra review produced to 'movie-reviews'.")

# ── 8b: Run the dry-run listener ──────────────────────────────────────────────
dry_run_listener = AGStream(
    atype=MovieSummary,
    input_topic="movie-reviews",
    output_topic="movie-summaries",
    schema_registry_url="http://localhost:8081",
    instructions=(
        "You receive a movie review. "
        "Produce a MovieSummary with a one-sentence summary and the overall sentiment."
    ),
)

print("Running dry-run listener (produce_results=False, auto_offset_reset='latest')...")

dry_run_listener.listen(
    source_atype_name="MovieReview",
    validate_schema=True,
    produce_results=False,         # <── transduce locally, do NOT write to Kafka
    verbose=True,
    auto_offset_reset="latest",    # <── only process messages produced after start
    max_empty_polls=5,             # exit after 5 consecutive empty polls
    schema_fetch_retries=5,
    schema_fetch_retry_delay=2.0,
)

print("\nDry-run complete — no messages were written to 'movie-summaries'.")

### Summary — `listen`

| Step | What happens |
|---|---|
| **Poll** | Continuously polls `input_topic` using a Kafka consumer |
| **Validate** | Each state dict is instantiated as `source_atype` (Pydantic validation) |
| **Transduce** | `asyncio.run(transducer.__lshift__(source_ag))` — LLM converts source → target |
| **Produce** | Transduced states are sent to `output_topic` via `produce` |
| **Stop** | `KeyboardInterrupt` (or Stop button) closes the consumer gracefully |

The method is the **streaming counterpart** of the batch `<<` operator: instead of
transducing a whole list at once, it processes one state at a time as it arrives on
the Kafka topic.

## 13. `aproduce_and_collect` — Async Fire-and-Collect

`aproduce_and_collect` is the **async counterpart** to the `listen` + `collect_sources`
pattern.  Instead of managing a background listener thread and a separate collector,
it does everything in one `await`:

1. **Produces** every state in `self.states` to `input_topic`, each tagged with a
   unique UUID key.
2. **Starts a background consumer** that polls `output_topic` and matches incoming
   messages by their Kafka key.
3. **Awaits** until every key has been collected or `timeout` seconds have elapsed.
4. **Returns** the results in the **same order** as the original `self.states` list.

### When to use it

| Scenario | Recommended method |
|---|---|
| Long-running daemon that processes an unbounded stream | `listen()` |
| One-shot batch: produce N items, collect N results, done | `aproduce_and_collect()` |

### Prerequisites

A `listen()` worker **must already be running** on the same `input_topic → output_topic`
pipeline.  `aproduce_and_collect` only produces and collects — it does **not** transduce.

### Key parameters

| Parameter | Default | Description |
|---|---|---|
| `result_atype` | `None` | Pydantic model class for the **output** (target) type — use when the output topic carries a different schema than the input topic |
| `timeout` | `120.0` | Max seconds to wait for all results |
| `poll_interval` | `0.5` | Seconds between output-topic poll cycles |
| `register_if_missing` | `True` | Auto-register source schema before producing |
| `verbose` | `False` | Print per-message progress to stderr |

### Step 1: Make sure the listener from Section 12 is still running

The listener_thread started in Section 12 (run_listener / listener.listen)
is a daemon thread, so it keeps running as long as the kernel is alive.
If you restarted the kernel, re-run the listener cell in Section 12 first.

In [ ]:
import asyncio
import threading

print(f"Listener thread alive: {listener_thread.is_alive()}")
if not listener_thread.is_alive():
    print("⚠️  Listener is not running — re-starting it now...")
    listener_thread = threading.Thread(target=run_listener, daemon=True)
    listener_thread.start()
    import time; time.sleep(3)
    print("🎧 Listener restarted.")

### Step 2: Build the producer AGStream

The producer is configured with:
  • atype         = MovieReview   (the type we are producing)
  • input_topic   = 'movie-reviews'   (where we send the raw reviews)
  • output_topic  = 'movie-summaries' (where we expect the transduced results)

aproduce_and_collect() will:
  1. Produce all states to 'movie-reviews' with unique UUID keys
  2. Poll 'movie-summaries' for messages whose key matches one of those UUIDs
  3. Return the results in the same order as sample_reviews

In [ ]:
pac_producer = AGStream(
    atype=MovieReview,
    input_topic="movie-reviews",
    output_topic="movie-summaries",
    schema_registry_url="http://localhost:8081",
)

pac_reviews = [
    MovieReview(
        title="Oppenheimer",
        review=(
            "A three-hour epic that never drags. Nolan's most mature work, "
            "blending historical drama with existential dread."
        ),
        rating=9.0,
    ),
    MovieReview(
        title="Barbie",
        review=(
            "Surprisingly subversive and visually inventive. "
            "Gerwig turns a toy commercial into a sharp feminist comedy."
        ),
        rating=8.1,
    ),
]

pac_producer.states = pac_reviews

print(f"Prepared {len(pac_reviews)} reviews for aproduce_and_collect:")
for r in pac_reviews:
    print(f"  • {r.title} (rating: {r.rating})")

### Step 3: Call aproduce_and_collect

In a Jupyter notebook the event loop is already running (managed by IPython /
nest_asyncio), so we use `await` directly.  In a plain Python script you would
use `asyncio.run(pac_producer.aproduce_and_collect(...))` instead.

In [ ]:
results = await pac_producer.aproduce_and_collect(
    result_atype=MovieSummary,
    register_if_missing=True,
    timeout=90.0,
    poll_interval=1.0,
    verbose=True,
)

print(f"\n📋 aproduce_and_collect returned {len(results)} result(s):")
for review, result_ag in zip(pac_reviews, results):
    if result_ag and result_ag.states:
        s = result_ag.states[0]
        print(f"\n  [{review.title}]")
        print(f"    Summary  : {s.one_line_summary}")
        print(f"    Sentiment: {s.sentiment}")
    else:
        print(f"\n  [{review.title}] ⚠️  No result received (timeout or error)")

### Summary — `aproduce_and_collect`

| Step | What happens |
|---|---|
| **Probe** | Records the current end-offset of `output_topic` to ignore stale messages |
| **Produce** | Sends all states to `input_topic` with unique UUID keys |
| **Consume** | Background thread polls `output_topic`, matches messages by key |
| **Await** | Async loop checks every `poll_interval` seconds until all keys collected or timeout |
| **Return** | Results list in the **same order** as the input states (`None` for timeouts) |

**Comparison with `listen` + `collect_sources`:**

```python
# ── Old pattern (manual orchestration)
producer.states = reviews
producer.produce(register_if_missing=True)
time.sleep(15)  # hope the listener is fast enough
results = collector.collect_sources(mode='all', max_messages=20)

# ── New pattern (aproduce_and_collect)
producer.states = reviews
results = await producer.aproduce_and_collect(timeout=90)
# results[i] corresponds exactly to reviews[i] — no guessing, no sleep()
```